In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd
import fiona
import os

os.environ["PROJ_LIB"] = r"C:\\Users\\SQwan\\miniconda3\\Library\\share"
import pickle
import googlemaps
import networkx as nx
import itertools
import csv
import glob
from collections import Counter
from datetime import datetime
from pprint import pprint
from mpl_toolkits.basemap import Basemap
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.distances import great_circle_distance_matrix
from shapely.geometry import Point, shape, Polygon
from shapely.ops import unary_union

import shapely.speedups

shapely.speedups.enable()

if not os.path.exists(r"Data"):
    os.makedirs(r"Data")

if not os.path.exists(r"Data\temp"):
    os.makedirs(r"Data\temp")

## Route stations

In [3]:
blue_station = [Point(42.116034, -86.452483),
                Point(42.115074, -86.465348),
                Point(42.110256, -86.463736),
                Point(42.110254, -86.457768),
                Point(42.101634, -86.448961),
                Point(42.101646, -86.441657),
                Point(42.102544, -86.436052),
                Point(42.088709, -86.437108),
                Point(42.084822, -86.437156),
                Point(42.080667, -86.434759),
                Point(42.085583, -86.433805),
                Point(42.085637, -86.424232),
                Point(42.082548, -86.421979),
                Point(42.082242, -86.418849),
                Point(42.077808, -86.424668),
                Point(42.102544, -86.436052),
                Point(42.107206, -86.446024),
                Point(42.109733, -86.447242),
                Point(42.116034, -86.452483),
                ]
blue_station = [Point(p.y, p.x) for p in blue_station]

red_station = [Point(42.101646,	-86.441657),
               Point(42.101634,	-86.448961),
               Point(42.116034,	-86.452483),
               Point(42.115074,	-86.465348),
               Point(42.111264,	-86.481872),
               Point(42.088810,	-86.478394),
               Point(42.084126,	-86.486379),
               Point(42.079074,	-86.493490),
               Point(42.033439,	-86.513542),
               Point(42.026502,	-86.516012),
               Point(42.086425,	-86.440537),
               Point(42.101646,	-86.441657),
               ]
red_station = [Point(p.y, p.x) for p in red_station]

yellow_station = [Point(42.118494913335645, -86.45082973186932),
                  Point(42.13082775201815, -86.4538851865351),
                  Point(42.13268958444188, -86.45128880811971),
                  Point(42.124573800847095, -86.4460383743168),
                  Point(42.121903066372475, -86.4390957589761),
                  Point(42.116026992072754, -86.4296080933503),
                  Point(42.11587877166418, -86.43641202669362),
                  Point(42.112791181420455, -86.4407060644722),
                  Point(42.10241413329736, -86.43602474092258),
                  Point(42.10241413, -86.43602474),
                  Point(42.11279118, -86.44070606),
                  Point(42.11587877, -86.43641203),
                  Point(42.11602699, -86.42960809),
                  Point(42.12190307, -86.43909576),
                  Point(42.1245738,	-86.44603837),
                  Point(42.13268958, -86.45128881),
                  Point(42.13082775	, -86.45388519),
                  Point(42.11849491, -86.45082973),
                  ]
yellow_station = [Point(p.y, p.x) for p in yellow_station]

In [10]:
brown_df = pd.read_csv(r"Data\new routes\brown_inbound1.csv", index_col=["id"])
brown_station = [Point(x, y) for x, y in zip(brown_df.xcoord, brown_df.ycoord)]

yellow_revised_df = pd.read_csv(
    r"Data\new routes\yellow_revised_inbound1.csv", index_col=["id"]
)
yellow_revised_station = [
    Point(x, y) for x, y in zip(yellow_revised_df.xcoord, yellow_revised_df.ycoord)
]

red_revised_df = pd.read_csv(
    r"Data\new routes\red_revised_inbound1.csv", index_col=["id"]
)
red_revised_station = [
    Point(x, y) for x, y in zip(red_revised_df.xcoord, red_revised_df.ycoord)
]

In [11]:
def locate_point(point, geoseries):
    """
    Checker function which determines if a coordinate is within 
    the geoseries and return the zone id where the point is 
    located
    """
    idx_list = geoseries.index[geoseries.geometry.contains(point)].tolist()
    return idx_list[0] if len(idx_list) >= 1 else np.nan

In [12]:
gdf = gpd.read_file(r"Data\shapefile\zone_id.shp")

In [14]:
blue_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in blue_station]
blue_station_id = [
    v
    for i, v in enumerate(blue_station_id)
    if i < len(blue_station_id) - 1
    if v != blue_station_id[i + 1]
]
blue_station_id += [gdf.loc[locate_point(blue_station[-1], gdf), "zone_id"]]

red_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in red_station]
red_station_id = [
    v
    for i, v in enumerate(red_station_id)
    if i < len(red_station_id) - 1
    if v != red_station_id[i + 1]
]
red_station_id += [gdf.loc[locate_point(red_station[-1], gdf), "zone_id"]]

yellow_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in yellow_station]
yellow_station_id = [
    v
    for i, v in enumerate(yellow_station_id)
    if i < len(yellow_station_id) - 1
    if v != yellow_station_id[i + 1]
]
yellow_station_id += [gdf.loc[locate_point(yellow_station[-1], gdf), "zone_id"]]

brown_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in brown_station]
brown_station_id = [
    v
    for i, v in enumerate(brown_station_id)
    if i < len(brown_station_id) - 1
    if v != brown_station_id[i + 1]
]
brown_station_id += [gdf.loc[locate_point(brown_station[-1], gdf), "zone_id"]]

yellow_revised_station_id = [
    gdf.loc[locate_point(p, gdf), "zone_id"] for p in yellow_revised_station
]
yellow_revised_station_id = [
    v
    for i, v in enumerate(yellow_revised_station_id)
    if i < len(yellow_revised_station_id) - 1
    if v != yellow_revised_station_id[i + 1]
]
yellow_revised_station_id += [
    gdf.loc[locate_point(yellow_revised_station[-1], gdf), "zone_id"]
]

red_revised_station_id = [
    gdf.loc[locate_point(p, gdf), "zone_id"] for p in red_revised_station
]
red_revised_station_id = [
    v
    for i, v in enumerate(red_revised_station_id)
    if i < len(red_revised_station_id) - 1
    if v != red_revised_station_id[i + 1]
]
red_revised_station_id += [
    gdf.loc[locate_point(red_revised_station[-1], gdf), "zone_id"]
]


In [15]:
red_revised_station_id

[17, 18, 13, 62, 64, 6, 50, 52, 50, 52, 29, 0, 40, 43]